In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

df1 = pd.read_csv('craigslistVehicles.csv')

A continuación, quería explorar mis datos. Al ejecutar el siguiente código, pude determinar que había 550313 filas y 22 columnas. De las 20 variables, decidí comenzar con solo algunas características y agregar gradualmente más para evitar un ajuste excesivo. 'Precio' fue mi variable de predicción (y), y 'año', 'fabricante', 'odómetro', 'marca', 'transmisión', 'lat', 'largo', 'combustible' fueron mis características iniciales. Usando .describe (), me di cuenta de que el precio y los odómetros de algunas publicaciones no tenían sentido: algunas publicaciones tenían un precio de 1 más de 1,000,000,000, y algunas tenían un kilometraje de más de 1,000,000 km. Lo resolví creando un umbral para 'precio' y 'odómetro'. Por último, eliminé las filas con valores nulos en cualquiera de las características elegidas.

(Nota: inicialmente comencé solo con 'año', 'fabricante' y 'odómetro', pero descubrí que agregar más variables mejoraba la precisión de mi modelo).

In [2]:
### Explorar Datos ### 
print(df1.columns)   # Muestra encabezados de columnas
print(len(df1.index)) # Muestra número de filas
print(len(df1.columns)) # Muestra número de columnas 

### Limpieza/Preparación de Datos ### 
# 1. Establecer un precio piso y techo
df1 = df1[df1['price'] < 99999.00]
df1 = df1[df1['price'] > 999.99]

# 2. Eliminar cualquier fila donde year esté en blanco o <1900
df1 = df1[df1['year'] > 1900]
df1 = df1.dropna(axis=0, subset=['year'])

# 3. Eliminar cualquier fila donde manufacturer esté en blanco
df1 = df1.dropna(axis=0, subset=['manufacturer'])

# 4. Eliminar cualquier fila donde el odometer esté en blanco o <999,999
df1 = df1.dropna(axis=0, subset=['odometer'])
df1 = df1[df1['odometer'] < 899999.00]

# 5. Eliminar cualquier fila donde make esté en blanco
df1 = df1.dropna(axis=0, subset=['make'])

# 6. Eliminar cualquier fila donde la transmission esté en blanco
df1 = df1.dropna(axis=0, subset=['transmission'])

# 7. Eliminar cualquier fila donde lat y long estén en blanco
df1 = df1.dropna(axis=0, subset=['lat'])
df1 = df1.dropna(axis=0, subset=['long'])

# 8. Eliminar cualquier fila donde fuel esté en blanco
df1 = df1.dropna(axis=0, subset=['fuel'])

Index(['url', 'city', 'city_url', 'price', 'year', 'manufacturer', 'make',
       'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'desc', 'lat', 'long'],
      dtype='object')
550313
22


Modelado de datos

Decidí usar un modelo de bosque aleatorio porque los bosques aleatorios son muy versátiles y también he tenido experiencia previa implementando uno en KNIME. Además, usé LabelEncoder () para poder usar datos categóricos como parte de mis valores X.

In [3]:
# Establecer el objetivo de predicción
y = df1.price

# Selección de características
used_car_features = ['year', 'odometer', 'manufacturer', 'make', 'transmission','lat','long','fuel']
X = df1[used_car_features]
# Necesito convertir string a float
le = preprocessing.LabelEncoder()
X = X.apply(le.fit_transform)

# Datos de entrenamiento y pruebas divididos
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# Definir modelo
used_car_model = RandomForestRegressor(random_state=1)

# Fit modelo
used_car_model.fit(train_X, train_y)

C:\Users\Usuario\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

Evaluación de datos 

A través de numerosas iteraciones de agregar más funciones (como se dijo antes), pude mejorar el error absoluto medio de 4375.07 a 1737. Como eso es todo! Con el tiempo, debería (con suerte) poder mejorar el MAE aún más.

In [4]:
### Evaluación de Datos ###
val_predictions = used_car_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))
print(df1['price'].mean())
print(len(df1.index)) # Ver número de filas

1787.6984376484772
14254.899043680107
359294
